In [55]:
using LinearAlgebra, SparseArrays;

In [56]:
dummy = Dict{String, Array{String}}("a"=>["b","c"], "b"=>["a"], "c"=>["a","d"], "d"=>["b"], "e"=>[])

Dict{String, Array{String, N} where N} with 5 entries:
  "c" => ["a", "d"]
  "e" => []
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [57]:
input=dummy

Dict{String, Array{String, N} where N} with 5 entries:
  "c" => ["a", "d"]
  "e" => []
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [58]:
damping=0.85
function naive_get_matrix(data)
    n=length(dummy)
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, value) in data
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        i=i+1
    end
    M=zeros(Float64, length(dummy), length(dummy))
    for (from, list) in data 
        l=length(list)
        if l>0 
            M[decodeNames[from], decodeNames[from]]=1-damping
            for to in list 
                M[decodeNames[to], decodeNames[from]]=damping/l
            end
        else
            M[decodeNames[from], decodeNames[from]]=1
        end
    end
    return (M, encodeNames, decodeNames)
end

naive_get_matrix (generic function with 1 method)

In [59]:
damping=0.85
function sparse_get_matrix(data)
    n=length(dummy)
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, value) in data
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        i=i+1
    end
    M=spzeros(Float64, length(dummy), length(dummy))
    for (from, list) in data 
        l=length(list)
        if l>0 
            M[decodeNames[from], decodeNames[from]]=1-damping
            for to in list 
                M[decodeNames[to], decodeNames[from]]=damping/l
            end
        else
            M[decodeNames[from], decodeNames[from]]=1
        end
    end
    return (M, encodeNames, decodeNames)
end

sparse_get_matrix (generic function with 1 method)

In [60]:
M, encode, decode=naive_get_matrix(input)

([0.15000000000000002 0.0 … 0.425 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.425 0.0 … 0.15000000000000002 0.0; 0.425 0.0 … 0.0 0.15000000000000002], ["c", "e", "b", "a", "d"], Dict("c" => 1, "e" => 2, "b" => 3, "a" => 4, "d" => 5))

In [61]:
M

5×5 Matrix{Float64}:
 0.15   0.0  0.0   0.425  0.0
 0.0    1.0  0.0   0.0    0.0
 0.0    0.0  0.15  0.425  0.85
 0.425  0.0  0.85  0.15   0.0
 0.425  0.0  0.0   0.0    0.15

In [62]:
function naive(input)
    M, encode, decode = naive_get_matrix(input)
    Λ, X=eigen(M)
    n=size(Λ,1)
    ranks=X[:, n]
    sites=map(r-> (encode[r[1]],Float64(r[2])), enumerate(ranks))
    return sort(sites, by= x->x[2], rev=true)
end

naive (generic function with 1 method)

In [63]:
r=naive(input)

5-element Vector{Tuple{String, Float64}}:
 ("a", 0.7302967433402213)
 ("b", 0.5477225575051661)
 ("c", 0.3651483716701108)
 ("d", 0.1825741858350554)
 ("e", 0.0)

In [64]:
function iterative(input, k)
    M, encode, decode = naive_get_matrix(input)
    n = size(M, 1)
    v = ones(n)/sqrt(n)
    for i in 1:k
        v=M*v
    end
    v=map(r-> (encode[r[1]],Float64(r[2])), enumerate(v))
    return sort(v, by= x->x[2], rev=true)
end

iterative (generic function with 1 method)

In [65]:
v=iterative(input, 40)

5-element Vector{Tuple{String, Float64}}:
 ("a", 0.7155417527999317)
 ("b", 0.5366563145999456)
 ("e", 0.4472135954999579)
 ("c", 0.35777087639997024)
 ("d", 0.17888543819998415)

In [66]:
function iterative_sparse(input, k)
    M, encode, decode = sparse_get_matrix(input)
    n = size(M, 1)
    v = ones(n)/sqrt(n)
    for i in 1:k
        v=M*v
    end
    v=map(r-> (encode[r[1]],Float64(r[2])), enumerate(v))
    return sort(v, by= x->x[2], rev=true)
end

iterative_sparse (generic function with 1 method)

In [67]:
v=iterative_sparse(input, 40)

5-element Vector{Tuple{String, Float64}}:
 ("a", 0.7155417527999317)
 ("b", 0.5366563145999456)
 ("e", 0.4472135954999579)
 ("c", 0.35777087639997024)
 ("d", 0.17888543819998415)

In [70]:
M, encode, decode = sparse_get_matrix(input)

(
 0.15    ⋅    ⋅    0.425   ⋅ 
  ⋅     1.0   ⋅     ⋅      ⋅ 
  ⋅      ⋅   0.15  0.425  0.85
 0.425   ⋅   0.85  0.15    ⋅ 
 0.425   ⋅    ⋅     ⋅     0.15, ["c", "e", "b", "a", "d"], Dict("c" => 1, "e" => 2, "b" => 3, "a" => 4, "d" => 5))